In [14]:
# !pip install Python==3.7.10
# !pip install spacy==2.1.4
# !pip install neuralcoref==4.0
# for version conflicts, check the following
# https://github.com/huggingface/neuralcoref
# https://github.com/explosion/spaCy/issues/4577

  Using cached spacy-2.3.5-cp37-cp37m-win_amd64.whl (9.5 MB)
  Using cached cymem-2.0.6-cp37-cp37m-win_amd64.whl (35 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached murmurhash-1.0.6-cp37-cp37m-win_amd64.whl (20 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached preshed-3.0.6-cp37-cp37m-win_amd64.whl (108 kB)
  Using cached importlib_metadata-4.8.1-py3-none-any.whl (17 kB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Using cached zipp-3.6.0-py3-none-any.whl (5.3 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\rob6460\\anaconda3\\envs\\ie\\lib\\site-packages\\numpy-1.21.3.dist-info\\METADATA'



  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\rob6460\\anaconda3\\envs\\ie\\lib\\site-packages\\numpy-1.21.3.dist-info\\METADATA'



In [1]:
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
# !python -m spacy download en
# pip install -U pip setuptools wheel
# pip install -U spacy
# !python -m spacy download en_core_web_sm
# test node
import en_core_web_sm
nlp = en_core_web_sm.load()

In [5]:
# Step 1: Coreference resolution
# find all expressions in the text that refer to a specific entity
# Neuralcoref model from Huggingface 
# SpaCy
import spacy
import neuralcoref


# Load SpaCy
# nlp = spacy.load('en')
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
#         print("debug")
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [6]:
# example text: “He” to “Elon Musk”
example_text = "Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."
example_text

"Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."

In [7]:
# text after coreference resolution
coref_text = coref_resolution(example_text)
coref_text

"Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla, Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 with 

In [11]:
# Step 2: Named Entity Linking
# using Named Entity Linking to construct a knowledge graph
# Facebook BLINK 32GB ram required
# so switch to old good Wikifier API
# split the text into sentences and remove the punctuations
import urllib
from string import punctuation
import nltk
import json
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [15]:
entities = wikifier(coref_text)
entities

[{'title': 'Elon Musk',
  'wikiId': 'Q317521',
  'label': 'Person',
  'characters': [(0, 8),
   (5, 8),
   (68, 76),
   (73, 76),
   (134, 142),
   (139, 142),
   (210, 218),
   (215, 218),
   (315, 318),
   (456, 459),
   (554, 562),
   (559, 562),
   (670, 678),
   (675, 678),
   (749, 757),
   (754, 757),
   (818, 826),
   (823, 826),
   (936, 944),
   (941, 944),
   (999, 1007),
   (1004, 1007),
   (1024, 1027)]},
 {'title': 'SpaceX',
  'wikiId': 'Q193701',
  'label': 'Organization',
  'characters': [(126, 131)]},
 {'title': 'Tesla, Inc.',
  'wikiId': 'Q478214',
  'label': 'Organization',
  'characters': [(198, 202), (198, 207), (198, 208)]},
 {'title': 'The Boring Company',
  'wikiId': 'Q28874479',
  'label': 'Organization',
  'characters': [(243, 260), (247, 260)]},
 {'title': 'Neuralink',
  'wikiId': 'Q29043471',
  'label': 'Organization',
  'characters': [(284, 292)]},
 {'title': 'Jeff Bezos',
  'wikiId': 'Q312556',
  'label': 'Person',
  'characters': [(444, 453), (449, 453)]}

In [16]:
import pandas as pd
pd.DataFrame(entities)

,title,wikiId,label,characters
0,Elon Musk,Q317521,Person,"[(0, 8), (5, 8), (68, 76), (73, 76), (134, 142..."
1,SpaceX,Q193701,Organization,"[(126, 131)]"
2,"Tesla, Inc.",Q478214,Organization,"[(198, 202), (198, 207), (198, 208)]"
3,The Boring Company,Q28874479,Organization,"[(243, 260), (247, 260)]"
4,Neuralink,Q29043471,Organization,"[(284, 292)]"
5,Jeff Bezos,Q312556,Person,"[(444, 453), (449, 453)]"
6,South Africa,Q258,Organization,"[(495, 507), (540, 551)]"
7,Pretoria,Q3926,Organization,"[(530, 537), (530, 551), (599, 606)]"
8,University of Pretoria,Q604444,Organization,"[(585, 606)]"
9,University of Pennsylvania,Q49117,Organization,"[(695, 724), (699, 724), (713, 724)]"


In [20]:
# Step 3: Relationship extraction
# Name Entity Recognition NER
# OpenNRE project； only infers relationships and doesn’t try to extract named entities
# https://github.com/thunlp/OpenNRE

# five open-source relationship extraction models that were trained on either the Wiki80 or Tacred dataset
# Wiki80 dataset is used
# wiki80_bert_softmax model (BERT encoder used)
# Iterate over every permutation pair of entities
import itertools
import opennre
relation_model = opennre.get_model('wiki80_bert_softmax')


for permutation in itertools.permutations(entities, 2):
    for source in permutation[0]['characters']:
        for target in permutation[1]['characters']:
            # Relationship extraction with OpenNRE
            data = relation_model.infer(
                {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
            if data[1] > relation_threshold:
                relations_list.append(
                    {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})


ModuleNotFoundError: No module named 'opennre'

In [ ]:
import json
import urllib
import pandas as pd
from neo4j import GraphDatabase

driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'letmein'))

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [102]:
def ie_pipeline(text, relation_threshold=0.9, entities_threshold=0.8):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("relation_threshold", relation_threshold),
        ("entities_threshold", entities_threshold)])
    
    url = "http://localhost:5000?" + data
    req = urllib.request.Request(url, data=data.encode("utf8"), method="GET")
    with urllib.request.urlopen(req, timeout=150) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    return response

## Elon Musk example input

In [89]:
example_data = ie_pipeline("""
Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX.
He is also early investor, CEO, and product architect of Tesla, Inc.
He is also the founder of The Boring Company and the co-founder of Neuralink. 
A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos.
Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa.
He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University.
He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics.
He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career.
He went on co-founding a web software company Zip2 with his brother Kimbal Musk.
  """)

In [90]:
print(example_data)

{'entities': [{'label': 'Organization', 'title': 'Tesla, Inc.', 'wikiId': 'Q478214'}, {'label': 'Person', 'title': 'Jeff Bezos', 'wikiId': 'Q312556'}, {'label': 'Organization', 'title': 'University of Pennsylvania', 'wikiId': 'Q49117'}, {'label': 'Person', 'title': 'Elon Musk', 'wikiId': 'Q317521'}, {'label': 'Organization', 'title': 'The Boring Company', 'wikiId': 'Q28874479'}, {'label': 'Organization', 'title': 'Stanford University', 'wikiId': 'Q41506'}, {'label': 'Person', 'title': 'Kimbal Musk', 'wikiId': 'Q6409751'}, {'label': 'Organization', 'title': 'Pretoria', 'wikiId': 'Q3926'}, {'label': 'Organization', 'title': 'University of Pretoria', 'wikiId': 'Q604444'}, {'label': 'Organization', 'title': 'SpaceX', 'wikiId': 'Q193701'}, {'label': 'Organization', 'title': 'Neuralink', 'wikiId': 'Q29043471'}], 'relations': [{'source': 'The Boring Company', 'target': 'Neuralink', 'type': 'subsidiary'}, {'source': 'Tesla, Inc.', 'target': 'Elon Musk', 'type': 'owned by'}, {'source': 'Elon Mu

In [91]:
import_direct_query = """
WITH $data as data
UNWIND data.entities as entity
MERGE (e:Entity{name:entity.title})
ON CREATE SET e.wikiId = entity.wikiId
WITH data, entity, e
CALL apoc.create.addLabels(e,[entity.label]) YIELD node
WITH data, count(*) as break_unwind
UNWIND data.relations as relation
MERGE (s:Entity{name:relation.source})
MERGE (t:Entity{name:relation.target})
WITH s,t,relation
CALL apoc.create.relationship(s, relation.type, {}, t) 
YIELD rel
RETURN distinct 'done'
"""

run_query(import_direct_query, {'data':example_data})

,'done'
0,done


## BBC news dataset example

Make sure to download the dataset from https://www.kaggle.com/hgultekin/bbcnewsarchive.

You will need to have the IE pipeline running on localhost:5000. You can run the following command to get it up and running:

<code>
    docker run -p 5000:5000 tomasonjo/trinityie
</code> 

In [93]:
run_query("MATCH (n) DETACH DELETE n")
run_query("CREATE CONSTRAINT IF NOT EXISTS ON (e:Entity) ASSERT e.name IS UNIQUE;")
run_query("CREATE INDEX rels IF NOT EXISTS FOR (n:Relation) ON (n.type);")

""


### Load the dataset

In [95]:
data = pd.read_csv('bbc-news-data.csv', delimiter='\t')
data.head()

In [97]:
import_refactored_query = """
UNWIND $params as value
CREATE (a:Article{content:value.content})
FOREACH (rel in value.ie.relations | 
  MERGE (s:Entity{name:rel.source})
  MERGE (t:Entity{name:rel.target})
  MERGE (s)-[:RELATION]->(r:Relation{type:rel.type})-[:RELATION]->(t)
  MERGE (a)-[:MENTIONS_REL]->(r))
WITH value, a
UNWIND value.ie.entities as entity
MERGE (e:Entity{name:entity.title})
SET e.wikiId = entity.wikiId
MERGE (a)-[:MENTIONS_ENT]->(e)
WITH entity, e
CALL apoc.create.addLabels(e,[entity.label]) YIELD node
RETURN distinct 'done'
"""

### Run 500 articles through IE pipeline and store results to Neo4j

In [103]:
with driver.session() as session:
    params = []
    for i,article in list(data.iterrows())[:500]:
        content = article['content']
        ie_data = ie_pipeline(content)
        params.append({'content':content, 'ie':ie_data})

        if (len(params) % 100 == 0):
            session.run(import_refactored_query, {'params':params})
            params = []

    session.run(update_query, {'params':params})